# 🚀 OnsetLab Agent Builder - Fine-tune Qwen 2.5 3B

**Model:** Qwen 2.5 3B Instruct (4-bit via Unsloth)

**What this does:**
1. Upload SDK zip + extract
2. Define tools (11 tools: memory + GitHub + Slack)
3. Generate training data (~366 examples, auto-calculated)
4. Fine-tune with LoRA (~15 min on T4)
5. Export GGUF for local deployment

---

## 1️⃣ Install Dependencies

In [ ]:
# Check GPU
!nvidia-smi --query-gpu=name --format=csv,noheader

# Install Unsloth (Colab-optimized)
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install training dependencies
!pip install -q --no-deps xformers trl peft accelerate bitsandbytes

# Install data generation dependencies
!pip install -q openai anthropic httpx

print('✅ All dependencies installed!')

In [ ]:
# Upload the SDK zip
from google.colab import files
print("Upload onsetlab.zip:")
uploaded = files.upload()

In [ ]:
# Extract and setup
!unzip -o onsetlab.zip

import sys
sys.path.insert(0, '.')

# Verify import
from onsetlab import AgentBuilder, BuildConfig, ToolSchema, MCPServerConfig
print('✅ SDK loaded successfully!')

## 2️⃣ Enter Your API Key

For training data generation. Choose ONE:
- **Anthropic**: https://console.anthropic.com/
- **OpenAI**: https://platform.openai.com/api-keys

In [ ]:
import os

# Option 1: Anthropic (Recommended)
os.environ['ANTHROPIC_API_KEY'] = ''  # @param {type:"string"}

# Option 2: OpenAI
os.environ['OPENAI_API_KEY'] = ''  # @param {type:"string"}

if os.environ.get('ANTHROPIC_API_KEY') or os.environ.get('OPENAI_API_KEY'):
    print('✅ API key configured!')
else:
    print('⚠️ Please enter an API key above')

## 3️⃣ Define Tools (11 tools)

Memory + GitHub + Slack tools. Edit to match your use case.

In [ ]:
#@title 📦 Tool Schemas (11 tools) - Click to expand
from onsetlab import ToolSchema

tools = [
    # === MEMORY TOOLS (4) ===
    ToolSchema(
        name="memory_set",
        description="Store a value in memory with a key. Use this to remember user preferences, important information, or context.",
        parameters={
            'key': {'type': 'string', 'description': "The key to store the value under (e.g., 'user_name', 'preferred_repo')"},
            'value': {'type': 'string', 'description': 'The value to store'}
        },
        required_params=['key', 'value'],
    ),
    ToolSchema(
        name="memory_get",
        description="Retrieve a stored value from memory by its key.",
        parameters={'key': {'type': 'string', 'description': 'The key to retrieve'}},
        required_params=['key'],
    ),
    ToolSchema(
        name="memory_delete",
        description="Remove a stored value from memory.",
        parameters={'key': {'type': 'string', 'description': 'The key to delete'}},
        required_params=['key'],
    ),
    ToolSchema(
        name="memory_list",
        description="List all keys currently stored in memory.",
        parameters={},
        required_params=[],
    ),
    
    # === GITHUB TOOLS (5) ===
    ToolSchema(
        name="issue_write",
        description="Create or update an issue in a repository",
        parameters={
            'owner': {'type': 'string', 'description': 'Repository owner'},
            'repo': {'type': 'string', 'description': 'Repository name'},
            'method': {'type': 'string', 'description': "Write operation: 'create' or 'update'"},
            'title': {'type': 'string', 'description': 'Issue title'},
            'body': {'type': 'string', 'description': 'Issue body content'},
            'issue_number': {'type': 'number', 'description': 'Issue number (required for update)'},
            'state': {'type': 'string', 'description': 'Issue state'},
            'assignees': {'type': 'array', 'description': 'Usernames to assign'},
            'labels': {'type': 'array', 'description': 'Labels to apply'}
        },
        required_params=['owner', 'repo', 'method'],
    ),
    ToolSchema(
        name="issue_read",
        description="Get issue details, comments, or sub-issues",
        parameters={
            'owner': {'type': 'string', 'description': 'Repository owner'},
            'repo': {'type': 'string', 'description': 'Repository name'},
            'issue_number': {'type': 'number', 'description': 'Issue number'},
            'method': {'type': 'string', 'description': "Read operation: 'get', 'get_comments', 'get_sub_issues'"},
        },
        required_params=['owner', 'repo', 'issue_number', 'method'],
    ),
    ToolSchema(
        name="list_issues",
        description="List issues in a repository",
        parameters={
            'owner': {'type': 'string', 'description': 'Repository owner'},
            'repo': {'type': 'string', 'description': 'Repository name'},
            'state': {'type': 'string', 'enum': ['OPEN', 'CLOSED', 'ALL'], 'description': 'Filter by state'},
            'labels': {'type': 'array', 'description': 'Filter by labels'},
        },
        required_params=['owner', 'repo'],
    ),
    ToolSchema(
        name="search_issues",
        description="Search for issues across repositories",
        parameters={
            'query': {'type': 'string', 'description': "Search query using GitHub syntax (e.g., 'is:open label:bug')"},
            'sort': {'type': 'string', 'description': 'Sort field'},
            'order': {'type': 'string', 'description': 'Sort order'},
        },
        required_params=['query'],
    ),
    ToolSchema(
        name="add_issue_comment",
        description="Add a comment to an issue",
        parameters={
            'owner': {'type': 'string', 'description': 'Repository owner'},
            'repo': {'type': 'string', 'description': 'Repository name'},
            'issue_number': {'type': 'number', 'description': 'Issue number'},
            'body': {'type': 'string', 'description': 'Comment text'},
        },
        required_params=['owner', 'repo', 'issue_number', 'body'],
    ),
    
    # === SLACK TOOLS (2) ===
    ToolSchema(
        name="conversations_add_message",
        description="Send a message to a Slack channel",
        parameters={
            'channel_id': {'type': 'string', 'description': "Channel name (e.g., '#general', '#engineering')"},
            'payload': {'type': 'string', 'description': 'Message text'},
            'thread_ts': {'type': 'string', 'description': 'Thread timestamp to reply in thread (optional)'},
        },
        required_params=['channel_id', 'payload'],
    ),
    ToolSchema(
        name="channels_list",
        description="List all channels in the Slack workspace",
        parameters={
            'channel_types': {'type': 'string', 'description': "Channel types: 'public_channel', 'private_channel'"},
            'limit': {'type': 'number', 'description': 'Maximum items to return'},
        },
        required_params=['channel_types'],
    ),
]

print(f'✅ Loaded {len(tools)} tools:')
for t in tools:
    print(f'   - {t.name}')

## 4️⃣ MCP Server Configs (2 servers)

In [ ]:
#@title 🔧 MCP Servers (2) - Click to expand
from onsetlab import MCPServerConfig

mcp_servers = [
    MCPServerConfig(
        package="ghcr.io/github/github-mcp-server",
        server_type="docker",
        docker_image="ghcr.io/github/github-mcp-server",
        auth_type="token",
        env_vars=["GITHUB_PERSONAL_ACCESS_TOKEN"],
        description="GitHub integration",
        tools=["issue_write", "issue_read", "list_issues", "search_issues", "add_issue_comment"],
    ),
    MCPServerConfig(
        package="korotovsky/slack-mcp-server",
        server_type="binary",
        auth_type="token",
        env_vars=["SLACK_MCP_XOXP_TOKEN"],
        description="Slack integration",
        tools=["conversations_add_message", "channels_list"],
    ),
]

print(f'✅ Configured {len(mcp_servers)} MCP servers')

## 5️⃣ Build Your Agent

This will:
1. Generate system prompt
2. Create **~730 training examples** (50/tool × 11 tools, auto-calculated)
3. Fine-tune Qwen 2.5 3B (~15-20 min)
4. Export GGUF for local use

In [ ]:
from onsetlab import AgentBuilder, BuildConfig
import os

# Build config - auto-calculates examples based on tool count
config = BuildConfig(
    num_examples=None,         # Auto-calculate: ~50/tool × 11 tools = ~730 total
    base_model='qwen2.5-3b',   # Qwen 2.5 3B Instruct
    epochs=None,               # Auto-adjust based on dataset size (2-3 for 700+)
    lora_rank=None,            # Auto-adjust: 24 for larger datasets
    learning_rate=None,        # Auto-adjust based on dataset size
    agent_name='my_agent',
    output_dir='./agent_build',
    save_gguf=True,            # Export GGUF for Ollama
    runtime='both',            # Generate Ollama + Python runtime
)

# Create builder
builder = AgentBuilder(
    problem_statement="Manage GitHub issues and send Slack notifications",
    tools=tools,
    mcp_servers=mcp_servers,
    api_key=os.environ.get('ANTHROPIC_API_KEY') or os.environ.get('OPENAI_API_KEY'),
    config=config,
)

print(f"\n🚀 Ready to build agent:")
print(f"   Tools: {len(tools)}")
print(f"   MCP Servers: {len(mcp_servers)}")
print(f"   Expected examples: ~{50 * len(tools) * 4 // 3} (50/tool auto-calculated)")

In [ ]:
# Build the agent! (Takes ~15-20 min total)
# - Data generation: ~5 min
# - Training: ~10-15 min
# - GGUF export: ~5 min

agent = builder.build()

## 6️⃣ Test the Model

In [ ]:
# Test cases
test_cases = [
    # Tool calls with correct params
    "List open issues in facebook/react",
    "Send 'Deploy complete!' to #engineering",
    "Remember my favorite repo is microsoft/vscode",
    "What's my favorite repo?",
    
    # Should ask for clarification
    "Create an issue",
    
    # Should NOT call a tool
    "Good morning!",
    "Thanks!",
    "What can you do?",
    
    # Should refuse
    "Book me a flight to NYC",
]

print("=== Testing Fine-tuned Model ===")
for query in test_cases:
    print(f"\nUser: {query}")
    # agent.chat() or test_query() depending on your implementation
    print("-" * 50)

## 7️⃣ Download Your Agent

In [ ]:
# Export and download
zip_path = agent.export()
print(f"\n📦 Agent exported to: {zip_path}")

from google.colab import files
files.download(zip_path)

## 📖 Usage Instructions

After downloading:

### Option 1: Ollama
```bash
# Unzip
unzip my_agent.zip
cd my_agent

# Create Ollama model
ollama create my-agent -f Modelfile

# Run
ollama run my-agent
```

### Option 2: Python Runtime
```bash
cd my_agent
pip install -r requirements.txt
python run_agent.py
```